<a href="https://colab.research.google.com/github/nlmaldonadog/GraficStruct/blob/master/Practica_1_Datos_de_entrada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 1: Datos de entrada
*Máster en Ciencia de Datos y Aprendizaje Automático - Universidad de La Rioja*

Durante el desarrollo de esta práctica, resuelve los ejercicios en este cuaderno justo después de ser propuestos.

Se valorará positivamente la documentación en las respuestas y la explicación de las mismas.

No olvides guardar tus versiones intermedias en GitHub (menú "Archivo" >> "Guardar una copia en GitHub") y la definitiva antes de la fecha límite de entrega.

Bibliografía:

* https://elitedatascience.com/imbalanced-classes
* Imbalanced: https://imbalanced-learn.org/stable/references/
* SMOTE: https://arxiv.org/abs/1106.1813
* Imblearn: https://imbalanced-learn.org/stable/references/generated/imblearn
* ADASYN: [artículo ADASYN](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fsci2s.ugr.es%2Fkeel%2Fpdf%2Falgorithm%2Fcongreso%2F2008-He-ieee.pdf)

##Datos balanceados y no balanceados

#### Conjunto de datos ¿balanceado?

Vamos a usar como entrada el conjunto de datos cuya descripción se encuentra en la siguiente URL "https://www.kaggle.com/code/mysticvalley/balance-scale-analysis-and-prediction".
Son unos datos sintéticos y los vamos a usar para entender que ocurre cuando los datos se encuentran balanceados o no.

Las variables L-Weight (var1), L-Distance (var2), R-Weight (var3) y R-Distance (var4) son numéricas y cada instancia ha sido clasificada de la siguiente manera
*   si var1\*var2=var3\*var4 entonces se clasifica como B,
*   si var1\*var2 > var3\*var4 entonces se clasifica como L
*   si var1\*var2 < var3\*var4 entonces se clasifica como R



Cargamos las librerías que vamos a utilizar:

In [1]:
import pandas as pd
import numpy as np

Para cargar los datos, accedemos al siguiente enlace de GitHub. También se podría hacer accediendo a la página de Kaggle, pero de esta manera evitamos el registro.

Mostramos los 5 primeros valores

In [2]:
# Leer dataset
url = "https://raw.githubusercontent.com/gadeamm/DataSets_AAII/main/balance-scale.csv"
df = pd.read_csv(url, sep=",")

# Mostrar las n primeras filas del dataset
df.head(5)

,Class,L-Weight,L-Distance,R-Weight,R-Distance
0,B,1,1,1,1
1,R,1,1,1,2
2,R,1,1,1,3
3,R,1,1,1,4
4,R,1,1,1,5


Vemos cuantas instancias hay de cada clase:

In [3]:
df['Class'].value_counts()

R    288
L    288
B     49
Name: Class, dtype: int64

Vemos que hay dos clases con el mismo número de datos (R y L) y otra que no (B).
Para simplificar y hacer aún más exagerado el desbalanceo, vamos a convertir este problema, en un problema de clasificación binaria.

Para ello vamos a re-etiquetar las instancias de la siguiente manera:
- las observaciones que son R y L serán etiquetadas como 0 (clase negativa)
- al resto de observaciones, las que están clasificadas como B, las re-etiquetaremos como 1 (clase positiva).

In [4]:
# Transformación del dataset a un conjunto binario
df['Class'] = [1 if b=='B' else 0 for b in df.Class]

# Mostrar el número de instancias para cada valor de la clase
df['Class'].value_counts()

0    576
1     49
Name: Class, dtype: int64

Como podemos ver, el 92% de la muestra es de la clase 0, esto quiere decir, que si **siempre** tuviéramos que predecir la clase 0, tendríamos un acierto del 92%.

El problema surge que nosotros no vamos a querer clasificar siempre una misma clase.

Veamos como afecta un conjunto de datos balanceado o no a la solución de un problema.

Para ello vamos a usar el algoritmo de Regresión Logística y el algoritmo k-NN (se explicarán con detenimiento más adelante) y la métrica de exactitud (o *accuracy*), todo de la librería Scikit-Learn.

In [6]:
#importamos las librerías correspondientes al algoritmo y la métrica
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

A la hora de usar el algoritmo de Regresión Logística (que no nos engañe el nombre, es un algoritmo de clasificación), vamos a usarlo con los parámetros que vienen por defecto, ya que nuestra intención ahora es ver la diferencia entre datos balanceados y sin balancear.

Más adelante trabajaremos sobre como separar los datos para entrenar, testear y validar, en este punto vamos a entrenar y testear con todo el conjunto de datos, ya que no vamos a hacer ajuste de hiperparámetros.

In [7]:
# Separamos las características (X) y la variable objetivo (y)
y = df.Class
X = df.drop('Class', axis=1)

# Entrenamos el modelo
model_lr = LogisticRegression().fit(X, y)
model_knn=KNeighborsClassifier().fit(X,y)

# Predecimos sobre el conjunto de entreanmiento
pred_y_lr = model_lr.predict(X)
pred_y_knn=model_knn.predict(X)

Muchos algoritmos están diseñados para maximizar la exactitud (*accuracy*) por defecto:

In [14]:
#¿Cómo es la accuracy?
print( accuracy_score(pred_y_lr, y) )
print( accuracy_score(pred_y_knn, y) )

0.9216
0.9136


Confirmamos así que el acierto de nuestro modelo es del 92% pero, ¿ocurre esto por qué sólo está prediciendo una clase?

In [19]:
# Mostramos los valores predichos (una vez cada valor)
print( np.unique( pred_y_lr ) )

[0]


De esta manera, comprobamos que este modelo sólo está prediciendo la clase 0, lo que significa que está ignorando la clase minoritaria a favor de la clase mayoritaria.

EJERCICIO: comprueba si el modelo knn predice sólo una clase o por el contrario predice de las dos.

In [20]:
print( np.unique( pred_y_knn ) )

[0 1]


#### Rta 0:
Para este caso el modelo de knn si esta prediciendo ambas clases.

Veamos dos técnicas para equilibrar esto:

##Remuestreo Aleatorio

####1. Aumentar la muestra de la clase minoritaria

El proceso de aumentar la muestra consiste en duplicar aleatoriamente las observaciones de la clase minoritaria para reforzar su señal.

Existen varias heurísticas para hacerlo, pero la forma más común es simplemente volver a muestrear con reemplazo.

Veamos como podemos hacerlo, pero primero cargamos el modulo *resample* de Scikit-Learn que vamos a usar:

In [21]:
from sklearn.utils import resample

El primer paso es crear un nuevo *DataFrame* con el nuevo conjunto de datos para la clase minoritaria. Para ello:
1. Separamos las instancias en grupos, un grupo por cada una de las clases que hay.
2. Re-muestreamos la clase minoritaria usando remplazamineto, y obteniendo el mismo número de muestras que la clase mayoritaria.
3. Finalmente, combinaremos en un nuevo *DataFrame* el nuevo grupo de datos de la clase que era minoritaria y el grupo original de la clase mayoritaria.
Para nuestro ejemplo, el código sería el siguiente:

In [22]:
# Separar las clases mayoritaria y minoritaria
df_majority = df[df.Class==0]
df_minority = df[df.Class==1]

# Aumentar la muestra de la clase minoritaria
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # muestra con remplazamiento
                                 n_samples=576,    # número de muestras de la clase mayoritaria
                                 random_state=123) # semilla para que los resultados sean reproducibles

# Combinar el nuevo grupo con el grupo original mayoritario
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Mostrar el número de instancias en cada clase
df_upsampled.Class.value_counts()

0    576
1    576
Name: Class, dtype: int64

Como podemos ver, el nuevo *DataFrame* tiene más instancias que el original y ahora, el ratio entre las dos clases es de 1:1.

Ahora, usando este conjunto de datos balanceado, vamos a entrenar un modelo de regresión logística.

In [23]:
# Separar las características (X) y la variable objetivo o clase (y)
y = df_upsampled.Class
X = df_upsampled.drop('Class', axis=1)

# Entrenar el modelo
model_lr = LogisticRegression().fit(X, y)

# Predecir en el conjunto de entrenamiento
pred_y_lr = model_lr.predict(X)

In [24]:
# Comprobamos cuáles y cuántas clases predice nuestro modelo
print( np.unique( pred_y_lr ) )

[0 1]


In [25]:
# ¿qué accuracy tenemos ahora?
print( accuracy_score(y, pred_y_lr) )

0.5147569444444444


Observamos que ahora nuestro modelo no predice sólo una clase, sin embargo es cierto que la *accuracy* ha disminuido.Pero tenemos que tener en cuenta que este valor es significativo de lo que está sucediendo, de cuál es el rendimiento del modelo.

EJERCICIO: Entrena un modelo de k-NN (con los hiperparámetros por defecto), y comprueba si clasifica de las dos clases y obtén la *accuracy* de dicho modelo.

In [26]:
model_knn=KNeighborsClassifier().fit(X,y)
pred_y_knn=model_knn.predict(X)
print( np.unique( pred_y_knn ) )
print( accuracy_score(y, pred_y_knn) )

[0 1]
0.9505208333333334


#### Rta 1:
Se puede observar que la accuracy es mucho mayor para el modelo de kNN, llegando hasta un 0,95, identificamos tambien que esta clasificando en las dos clases.

####2. Disminuir la muestra de la clase mayoritaria

Disminuir el número de instancias de la clase mayoritaria implica la eliminación aleatoria de observaciones para evitar que su señal domine el algoritmo de aprendizaje.

La heurística más común para hacerlo es el remuestreo sin reemplazo.

El proceso es similar al de muestreo visto anteriormente:

1. Separamos las instancias de cada clase en diferentes *DataFrame*.
2. Volvemos a muestrear la clase mayoritaria, esta vez sin remplazo y estableciendo el número de muestras para que coincida con el de la clase minoritaria.
3. Finalmente, combinaremos el nuevo *DataFrame* de la clase mayoritaria con el *DataFrame* original de la clase minoritaria.

In [27]:
# Separar las clases en diferentes DataFrame
df_majority = df[df.Class==0]
df_minority = df[df.Class==1]

# Disminuir la clase mayoritaria
df_majority_downsampled = resample(df_majority,
                                 replace=False,    # muestra sin remplazamiento
                                 n_samples=49,     # número de muestras de la clase minoritaria
                                 random_state=123) # semilla para que los datos sean reproducibles

# Combinar el nuevo grupo con el grupo original minoritario
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Mostrar el número de instancias en cada clase
df_downsampled.Class.value_counts()

0    49
1    49
Name: Class, dtype: int64

Esta vez, el nuevo *DataFrame* ha disminuido el número de instancias con respecto a su original, y el ratio entre las dos clases, vuelve a ser de 1:1.

Veamos ahora que ocurre si entrenamos con el algoritmo de regresión logística.

In [28]:
# Separar las características (X) y la variable objetivo o clase (y)
y = df_downsampled.Class
X = df_downsampled.drop('Class', axis=1)

# Entrenar el modelo
clf_2 = LogisticRegression().fit(X, y)

# Predecir en el conjunto de entrenamiento
pred_y_2 = clf_2.predict(X)

In [29]:
# Comprobamos cuáles y cuántas clases predice nuestro modelo
print( np.unique( pred_y_2 ) )

[0 1]


In [30]:
# ¿qué accuracy tenemos ahora?
print( accuracy_score(y, pred_y_2) )

0.5612244897959183


Con esta técnica hemos vuelto a evitar que el modelo prediga sólo una clase y además parece que la *accuracy* es más alta.

Veamos ahora otra técnica para balancear nuestras muestras.

EJERCICIO: Entrena un modelo de k-NN (con los hiperparámetros por defecto), y comprueba si clasifica de las dos clases y obtén la *accuracy* de dicho modelo.

In [31]:
model_knn_2=KNeighborsClassifier().fit(X,y)
pred_y_knn_2=model_knn_2.predict(X)
print( np.unique( pred_y_knn_2 ) )
print( accuracy_score(y, pred_y_knn_2) )

[0 1]
0.7959183673469388


#### Rta 2:
En este caso podemos observar el efectivamente si esta prediciendo ambas clases, pero, el accuracy se redujo mucho comparando con el ejercicio anterior. Sigue siendo mejor comparado contra el algoritmo de regresion logistica, pero el accuracy no mejora de la misma manera que al aumentar la clase minoritaria, por lo menos en este ejemplo

####3. Combinar ambas técnicas: aumentar y disminuir muestras

EJERCICIO: La clase mayoritaria tiene en nuestro ejemplo 576 instancias y la clase minoritaria 49, vamos a balancear el conjunto de datos aumentando la clase minoritaria y disminuyendo la clase mayoritaria a 312 muestras. Elegimos este número de instancias porque de esta manera hacemos que el número de muestras que se generan y que se eliminan en cada clase es el mismo.

Comprueba que accuracy saldría para los modelos de Regresión Logística y k-NN.
¿Qué modelo da mejor resultado?

In [32]:
# Separar las clases mayoritaria y minoritaria
df_majority_e3 = df[df.Class==0]
df_minority_e3 = df[df.Class==1]

# Aumentar la muestra de la clase minoritaria a 312
df_minority_upsampled_e3 = resample(df_minority_e3,
                                 replace=True,     # muestra con remplazamiento
                                 n_samples=312,    # número de muestras de la clase mayoritaria
                                 random_state=123) # semilla para que los resultados sean reproducibles

# Disminuir la clase mayoritaria a 312
df_majority_downsampled_e3 = resample(df_majority_e3,
                                 replace=False,    # muestra sin remplazamiento
                                 n_samples=312,     # número de muestras de la clase minoritaria
                                 random_state=123) # semilla para que los datos sean reproducibles

df_resampled_comb_e3 = pd.concat([df_majority_downsampled_e3, df_minority_upsampled_e3])
df_resampled_comb_e3.Class.value_counts()

0    312
1    312
Name: Class, dtype: int64

In [34]:
# Separar las características (X) y la variable objetivo o clase (y)
y_3 = df_resampled_comb_e3.Class
X_3 = df_resampled_comb_e3.drop('Class', axis=1)

# Entrenar el modelo LR y kNN
model_lr_e3 = LogisticRegression().fit(X_3, y_3)
pred_y_lr_e3 = model_lr_e3.predict(X_3)
model_knn_e3 = KNeighborsClassifier().fit(X_3,y_3)
pred_y_knn_3 = model_knn_e3.predict(X_3)

print( "Clases que esta prediciendo el algoritmo de regresion logistica", np.unique( pred_y_lr_e3 ) )
print( "Accuracy del algoritmo de regresion logistica", accuracy_score(y_3, pred_y_lr_e3) )
print( "Clases que esta prediciendo el algoritmo de kNN", np.unique( pred_y_knn_3 ) )
print( "Accuracy del algoritmo de kNN", accuracy_score(y_3, pred_y_knn_3) )

Clases que esta prediciendo el algoritmo de regresion logistica [0 1]
Accuracy del algoritmo de regresion logistica 0.530448717948718
Clases que esta prediciendo el algoritmo de kNN [0 1]
Accuracy del algoritmo de kNN 0.907051282051282


#### Rta 3:
Al combinar las tenicas podemos ver que para este problema en particular el algoritmo de kNN sigue siendo bastantemente mejor, en este caso tenemos una mejora sobre el caso de disminuir la muestra, pero el accuracy entregado por el algoritmo kNN a la hora de aumentar por completo la clase minoritaria sigue siendo mejor

EJERCICO: Visto que ocurre aplicando cada técnica por separado y combinándolas ¿qué número de muestra crees que será una mejor elección para nuestro ejemplo?
- aquella que requiera que el número de muestras de oversampling sea mayor (por ejemplo si el conjunto lo tomáramos de 400 instancias para cada clase)
- o aquella que requiera que el número de muestras de oversampling sea menor (por ejemplo, con 100 instancias para cada clase).

Probar si nuestra hipótesis es cierta para este ejemplo. ¿Qué ocurre?

In [40]:
# Separar las clases mayoritaria y minoritaria
df_majority_4 = df[df.Class==0]
df_minority_4 = df[df.Class==1]


df_minority_upsampled_4a = resample(df_minority_4,
                                 replace=True,     # muestra con remplazamiento
                                 n_samples=400,    # número de muestras de la clase mayoritaria
                                 random_state=123) # semilla para que los resultados sean reproducibles
df_majority_downsampled_4a = resample(df_majority_4,
                                 replace=False,    # muestra sin remplazamiento
                                 n_samples=400,    # número de muestras de la clase minoritaria
                                 random_state=123) # semilla para que los datos sean reproducibles

df_minority_upsampled_4b = resample(df_minority_4,
                                 replace=True,     # muestra con remplazamiento
                                 n_samples=100,    # número de muestras de la clase mayoritaria
                                 random_state=123) # semilla para que los resultados sean reproducibles
df_majority_downsampled_4b = resample(df_majority_4,
                                 replace=False,    # muestra sin remplazamiento
                                 n_samples=100,    # número de muestras de la clase minoritaria
                                 random_state=123) # semilla para que los datos sean reproducibles

df_resampled_comb_4a = pd.concat([df_majority_downsampled_4a, df_minority_upsampled_4a])
df_resampled_comb_4b = pd.concat([df_majority_downsampled_4b, df_minority_upsampled_4b])
print("Conteo primera parte del ejercicio")
print(df_resampled_comb_4a.Class.value_counts())
y_4a = df_resampled_comb_4a.Class
X_4a = df_resampled_comb_4a.drop('Class', axis=1)
# Entrenar el modelo LR y kNN
model_lr_4a = LogisticRegression().fit(X_4a, y_4a)
pred_y_lr_4a = model_lr_4a.predict(X_4a)
model_knn_4a = KNeighborsClassifier().fit(X_4a,y_4a)
pred_y_knn_4a = model_knn_4a.predict(X_4a)

print( "Clases que esta prediciendo el algoritmo de regresion logistica para 400 muestras", np.unique( pred_y_lr_4a ) )
print( "Accuracy del algoritmo de regresion logistica para 400 muestras", accuracy_score(y_4a, pred_y_lr_4a) )
print( "Clases que esta prediciendo el algoritmo de kNN para 400 muestras", np.unique( pred_y_knn_4a ) )
print( "Accuracy del algoritmo de kNN para 400 muestras", accuracy_score(y_4a, pred_y_knn_4a) )

print("Conteo segunda parte del ejercicio")
print(df_resampled_comb_4b.Class.value_counts())
y_4b = df_resampled_comb_4b.Class
X_4b = df_resampled_comb_4b.drop('Class', axis=1)
# Entrenar el modelo LR y kNN
model_lr_4b = LogisticRegression().fit(X_4b, y_4b)
pred_y_lr_4b = model_lr_4b.predict(X_4b)
model_knn_4b = KNeighborsClassifier().fit(X_4b,y_4b)
pred_y_knn_4b = model_knn_4b.predict(X_4b)

print( "Clases que esta prediciendo el algoritmo de regresion logistica para 100 muestras", np.unique( pred_y_lr_4b ) )
print( "Accuracy del algoritmo de regresion logistica para 100 muestras", accuracy_score(y_4b, pred_y_lr_4b) )
print( "Clases que esta prediciendo el algoritmo de kNN para 100 muestras", np.unique( pred_y_knn_4b ) )
print( "Accuracy del algoritmo de kNN para 100 muestras", accuracy_score(y_4b, pred_y_knn_4b) )

Conteo primera parte del ejercicio
0    400
1    400
Name: Class, dtype: int64
Clases que esta prediciendo el algoritmo de regresion logistica para 400 muestras [0 1]
Accuracy del algoritmo de regresion logistica para 400 muestras 0.49625
Clases que esta prediciendo el algoritmo de kNN para 400 muestras [0 1]
Accuracy del algoritmo de kNN para 400 muestras 0.9275
Conteo segunda parte del ejercicio
0    100
1    100
Name: Class, dtype: int64
Clases que esta prediciendo el algoritmo de regresion logistica para 100 muestras [0 1]
Accuracy del algoritmo de regresion logistica para 100 muestras 0.55
Clases que esta prediciendo el algoritmo de kNN para 100 muestras [0 1]
Accuracy del algoritmo de kNN para 100 muestras 0.765


### Rta 4:
Posterior a realizar estas ejecuciones podemos identificar que para el caso de esta muestra, el algoritmo de regresion logistica empeora entre mas datos se agregan a las muestras, por caso contrario, el algoritmo de kNN mejora conforme mas muestras de la clase minoritaria se incrementa con la finalidad de igualar a la clase mayoritaria

EJERCICIO EXTRA: existe otra librería que incluye estos métodos de balanceo de datos [*imblearn*](https://imbalanced-learn.org/stable/references/) y los métodos se definen como *RandomOverSampler* y *RandomUnderSampler*. Lee como se usa y prueba a usarlos como hemos hecho con los métodos de la librería Scikit-Learn (oversampling, undersampling, y combinado). Obtén la *accuracy* para al menos uno de los modelos (LR y/o k-NN).


### SMOTE: Synthetic Minority Oversampling TEchnique

Como hemos visto, una manera de solucionar el problema de que una clase tenga menos muestras que otra, es duplicar las instancias de la clase minoritaria en el conjunto de entrenamiento. Así obtenemos un conjunto balanceado pero esta solución no proporciona información adicional al modelo.

En lugar de duplicar las instancias de la clase minoritaria podemos *sintetizar* (crear de manera artificial) nuevas instancias que pertenezcan a esta muestra.

Quizás una de las técnicas más usadas es la que se denomina **Synthetic Minority Oversampling TEchnique** (SMOTE). Esta técnica fue descrita por Nitesh Chawla et al. en 2002 en el artículo "[SMOTE: Synthetic Minority Oversampñling Technique](https://arxiv.org/abs/1106.1813)".

SMOTE funciona seleccionando ejemplos cercanos en el espacio de características, dibujando una línea entre los ejemplos en el espacio de características y dibujando una nueva muestra en un punto a lo largo de esa línea.

Específicamente, primero se elige un ejemplo aleatorio de la clase minoritaria. Entonces se encuentran k de los vecinos más cercanos para ese ejemplo (típicamente k = 5). Se elige un vecino seleccionado al azar y se crea un ejemplo sintético en un punto seleccionado al azar entre los dos ejemplos en el espacio de características.
Este procedimiento se puede utilizar para crear tantos ejemplos sintéticos para la clase minoritaria como sean necesarios. Como se describe en el documento, sugiere usar primero un submuestreo aleatorio para recortar el número de ejemplos en la clase mayoritaria, luego usar SMOTE para sobremuestrear la clase minoritaria para equilibrar la distribución de clases.

A continuación vemos un ejemplo de uso , y para ello vamos a necesitar la librería *imbalanced-learn*

In [ ]:
# Comprobamos que está instalada y cuál es su versión
import imblearn
print(imblearn.__version__)
from imblearn.over_sampling import SMOTE

Importamos el resto de librerías que se van a usar

In [ ]:
# Resto de librerías que vamos a usar
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where

Definimos un conjunto de datos aleatorio con dos clases. Para ello usamos la instrucción *make_classification* del módulo datasets de la librería Scikit-Learn.

Lo que hace este método es crear conjuntos de datos multiclase, asignando a cada clase una o más instancias distribuidas normalmente. Este método está especializado en introducir ruido por medio de las muestras, es decir, genera caracterísitcas correlacionas, redundantes y no informativas.


In [ ]:
# Creamos el conjunto de datos aleatorio
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
	n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)
# Mostramos como está distribuida la variable objetivo
counter = Counter(y)
print(counter)

Dibujamos como sería nuestro dataset

In [ ]:
# Dibujamos en un gráfico el dataset creado
for label, _ in counter.items():
	row_ix = where(y == label)[0]
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

Aplicamos la técnica SMOTE:

In [ ]:
# Transformamos el conjunto de datos
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
# Obtenemos la nueva distribución de clases en la variable y
counter = Counter(y)
print(counter)

In [ ]:
# # Dibujamos en un gráfico el dataset transformado
for label, _ in counter.items():
	row_ix = where(y == label)[0]
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

El artículo original sobre SMOTE sugiere combinar SMOTE con un submuestreo aleatorio de la clase mayoritaria. La biblioteca de aprendizaje desequilibrado admite submuestreo aleatorio a través de la clase RandomUnderSampler.

Podemos actualizar el ejemplo para sobremuestrear primero la clase minoritaria para tener un 10 por ciento del número de ejemplos de la clase mayoritaria (por ejemplo, alrededor de 1000), luego usar un submuestreo aleatorio para reducir el número de ejemplos en la clase mayoritaria para tener un 50 por ciento más que el clase minoritaria (por ejemplo, alrededor de 2000).

Para implementar esto, podemos especificar las proporciones deseadas como argumentos para las clases SMOTE y RandomUnderSampler;

Volvemos a definir el conjunto de datos original.

In [ ]:
# define dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
	n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)
# summarize class distribution
counter = Counter(y)
print(counter)

A continuación, se transforma el conjunto de datos, primero sobremuestreando la clase minoritaria y luego submuestreando la clase mayoritaria.

La distribución de clases final después de esta secuencia de transformaciones coincide con una proporción de 1: 2 o aproximadamente 2000 ejemplos en la clase mayoritaria y aproximadamente 1000 ejemplos en la clase minoritaria.

In [ ]:
# check version number
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
# define pipeline
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X, y = pipeline.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)


Nota: aquí puedes encontrar más información de cómo usar [*pipeline*.](https://imbalanced-learn.org/stable/references/generated/imblearn.pipeline.Pipeline.html)

Mostramos el resultado mediante un *ScatterPlot* o gráfico de puntos.

In [ ]:
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y == label)[0]
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

###ADASYN: Adaptative Synthetic Sampling Approach For Imbalanced Learning

Otro enfoque implica la generación de muestras sintéticas inversamente proporcional a la densidad de los ejemplos de la clase minoritaria.

Es decir, generar más ejemplos sintéticos en regiones del espacio de características donde la densidad de ejemplos minoritarios es menor que la densidad de la clase mayoritaria.

Esta modificación de SMOTE se conoce como Método de muestreo sintético adaptativo, o ADASYN, y lo propuso a Haibo He, et al. en su artículo de 2008 llamado así por el método titulado “[ADASYN: Adaptive Synthetic Sampling Approach For Imbalanced Learning.](https://sci2s.ugr.es/keel/pdf/algorithm/congreso/2008-He-ieee.pdf)”.

La idea clave del algoritmo ADASYN es utilizar una distribución de densidad como criterio para decidir automáticamente el número de muestras sintéticas que deben generarse para cada ejemplo de datos minoritarios.

Podemos aplicar este procedimiento utilizando la clase ADASYN en la librería *imbalanced-learn*.

Cargamos las librerías que vamos a necesitar:

In [ ]:
# Oversample and plot imbalanced dataset with ADASYN
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import ADASYN
from matplotlib import pyplot
from numpy import where

Volvemos a crear un conjunto de datos de dos clases con dos características para usarlo como ejemplo.

In [ ]:
# define dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
	n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)
# summarize class distribution
counter = Counter(y)
print(counter)

Usamos el método ADASYN que está en la librería imbalanced-learn.

In [ ]:
# transform the dataset
oversample = ADASYN()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

In [ ]:
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y == label)[0]
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

EJERCICIO: Tanto el método SMOTE y ADASYN se basan, para sintetizar nuevas muestras, en el estudio de los k vecinos de cada una de las instancias. Por defecto se ha comentado que ambos métodos usan k=5. Prueba a ver que ocurre para ambos métodos, con k=3 y con k=7, ¿cambia mucho el conjunto final según el número de vecinos que usemos?

EJERCICIO:

Aplica las técnicas de Aumentar la muestra (Oversampling), Disminuir la muestra (Undersampling), SMOTE y ADASYN sobre el conjunto de datos que se encuentra en [https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv ](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv)

Este conjunto de datos es original del *National Institute of Diabetes and Digestive and Kidney Diseases* y se usa para predecir si un paciente tiene diabetes o no, basándose en las medidas de diagnóstico que se recogen en los datos.

El nombre de las columnas del conjunto de datos son ['NO_PREG','PLASMA_GLUCOSE','DIASTOLIC_BP','SKIN_THICKNESS','SERUM_INSULIN','BMI','DIA_PEDI_FUNC','AGE','TARGET'] siendo la última, la característica objetivo.

EJERCICIO EXTRA: Una vez tengas los conjuntos balanceados para los cuatro tipos de métodos, calcula la *accuracy* que sale al aplicar el método de Regresión Logística sobre cada conjunto, teniendo en cuenta que el valor de la *accuracy* ha de estar entre 0 y 1 siendo 1 el mejor resultado: ¿Hay alguno que sea claramente mejor?

EJERCICIO EXTRA: En clase de teoría se ha explicado que una estrategia para balancear una clase y de esta manera poder eliminar ruido del dataset (posibles datos anómalos) es usar el método *One-Class Classification* (Clasificación de una clase).

Crea una conjunto de datos aleatorio con 10000 muestras, dónde de la clase minoritaria sólo haya 10, siendo la distribución de la muestra 1:1000.

La librería *scikit-learn* tiene implementado varios algoritmos de clasificación para una clase, en particular:


*   One-class SVM
*   Isolation Forest

Busca como usar l menos uno de estos dos métodos y ponlo en práctica en el conjunto de datos generado.

